Import Dependencies

In [45]:
import kotlin.math.*
import kotlin.random.Random
import kotlin.math.abs
import kotlin.math.ceil

Generate Original Forcast

In [46]:
val forecast = listOf(32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.30)
println("Original forecast: $forecast")
println("Number of days: ${forecast.size}")

Original forecast: [32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.3]
Number of days: 14


Takes one input (losing threshold), uses Knuth Algorithm

In [47]:
// Test the Poisson function
fun generatePoisson(forecast: List<Double>): Any {
    return forecast.map { lambda ->
        val L = exp(-lambda)
        var k = 0
        var p = 1.0

        do {
            k++
            p *= Random.nextDouble()
        } while (p > L)

        return k - 1
    }
}

val forecast = listOf(32.83,26.12,23.48,22.57,23.14,24.68,26.13,28.89,23.35,20.99,20.18,20.68,22.04,23.30)
val result = generatePoisson(5.0)
println("Poisson test result: $result")

Poisson test result: 7


- Simulate runs the forecast instead of single number.
- Generate binomial flips a coin but with a bias. Forecast values must always be greater than number of trials
- Simulate binomial does the same but with every number in forecast

In [48]:
// Test the complete simulation functions
fun simulatePoissonDemand(forecast: List<Double>): List<Int> {
    return forecast.map { lambda ->
        generatePoisson(lambda.toDouble())
    }
}

fun generateBinomial(n: Int, p: Double): Int {
    var count = 0
    repeat(n) {
        if (Random.nextDouble() < p) {
            count++
        }
    }
    return count
}

fun simulateBinomialDemand(forecast: List<Double>, trials: Int = 50): List<Int> {
    return forecast.map { f ->
        val probability = minOf(f.toDouble() / trials, 1.0)
        generateBinomial(trials, probability)
    }
}

// Test them
val forecast = listOf(32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.30)

val poissonResult = simulatePoissonDemand(forecast)
println("Poisson simulation results:")
println("Forecast: $forecast")
println("Poisson:  $poissonResult")

val binomialResult = simulateBinomialDemand(forecast, trials = 50)
println("\nBinomial simulation results:")
println("Forecast: $forecast")
println("Binomial: $binomialResult")

Poisson simulation results:
Forecast: [32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.3]
Poisson:  [33, 26, 23, 22, 19, 24, 26, 19, 23, 21, 17, 20, 20, 25]

Binomial simulation results:
Forecast: [32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.3]
Binomial: [36, 19, 23, 25, 13, 28, 28, 21, 19, 25, 22, 15, 28, 20]


In [51]:
fun compareSimulations(forecast: List<Double>, trials: Int = 50, numRuns: Int = 3) {
    println("\nComparing Poisson vs Binomial simulations ($numRuns runs):")
    println("Forecast: $forecast")
    println("-".repeat(80))

    repeat(numRuns) { run ->
        val poissonResult = simulatePoissonDemand(forecast)
        val binomialResult = simulateBinomialDemand(forecast, trials)

        println("Run ${run + 1}:")
        println("  Poisson:  $poissonResult")
        println("  Binomial: $binomialResult")
        println()
    }
}

// Test it
val forecast = listOf(32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.30)
compareSimulations(forecast, trials = 50, numRuns = 3)


Comparing Poisson vs Binomial simulations (3 runs):
Forecast: [32.83, 26.12, 23.48, 22.57, 23.14, 24.68, 26.13, 28.89, 23.35, 20.99, 20.18, 20.68, 22.04, 23.3]
--------------------------------------------------------------------------------
Run 1:
  Poisson:  [40, 30, 20, 18, 33, 26, 24, 37, 20, 28, 10, 24, 16, 23]
  Binomial: [33, 21, 15, 18, 20, 27, 23, 26, 25, 16, 23, 23, 19, 22]

Run 2:
  Poisson:  [29, 30, 28, 22, 16, 31, 21, 39, 33, 29, 22, 14, 16, 24]
  Binomial: [34, 29, 26, 24, 22, 26, 30, 34, 25, 18, 22, 20, 20, 29]

Run 3:
  Poisson:  [26, 14, 26, 29, 24, 23, 19, 21, 19, 23, 21, 22, 19, 17]
  Binomial: [36, 28, 23, 25, 29, 20, 28, 34, 24, 19, 15, 21, 25, 27]



In [59]:
// Extension function for standard deviation (needed for analysis)
fun List<Int>.standardDeviation(): Double {
    val mean = this.average()
    val variance = this.map { (it - mean) * (it - mean) }.average()
    return kotlin.math.sqrt(variance)
}

// Lighter analysis function (fewer runs for notebook)
fun lightAnalyzeSimulations(forecast: List<Double>, trials: Int = 10, numRuns: Int = 100) {
    println("\nLight Statistical Analysis ($numRuns runs):")
    println("=".repeat(50))

    // Store results
    val poissonErrors = mutableListOf<Double>()
    val binomialErrors = mutableListOf<Double>()
    val poissonTotals = mutableListOf<Int>()
    val binomialTotals = mutableListOf<Int>()

    // Run simulations
    repeat(numRuns) {
        val poissonResult = simulatePoissonDemand(forecast)
        val binomialResult = simulateBinomialDemand(forecast, trials)

        // Calculate errors
        val poissonError = poissonResult.zip(forecast) { p, f -> kotlin.math.abs(p - f) }.sum()
        val binomialError = binomialResult.zip(forecast) { b, f -> kotlin.math.abs(b - f) }.sum()

        poissonErrors.add(poissonError)
        binomialErrors.add(binomialError)
        poissonTotals.add(poissonResult.sum())
        binomialTotals.add(binomialResult.sum())
    }

    // Calculate and display results
    val forecastTotal = forecast.sum()

    println("Original forecast total: $forecastTotal")
    println()

    println("POISSON RESULTS:")
    println("  Average total demand: ${"%.2f".format(poissonTotals.average())}")
    println("  Average absolute error: ${"%.2f".format(poissonErrors.average())}")

    println()

    println("BINOMIAL RESULTS:")
    println("  Average total demand: ${"%.2f".format(binomialTotals.average())}")
    println("  Average absolute error: ${"%.2f".format(binomialErrors.average())}")

    println()

    // Comparison
    val binomialBetter = poissonErrors.zip(binomialErrors) { p, b -> b < p }.count { it }
    val poissonBetter = numRuns - binomialBetter

    println("COMPARISON:")
    println("  Binomial more accurate: $binomialBetter/$numRuns times (${"%.1f".format(binomialBetter.toDouble() / numRuns * 100)}%)")
    println("  Poisson more accurate: $poissonBetter/$numRuns times (${"%.1f".format(poissonBetter.toDouble() / numRuns * 100)}%)")

    if (poissonErrors.average() < binomialErrors.average()) {
        println("  Winner: Poisson (lower average error)")
    } else {
        println("  Winner: Binomial (lower average error)")
    }
}

// Test the analysis
val forecast = listOf(32.83,26.12,23.48,22.57,23.14,24.68,26.13,28.89,23.35,20.99,20.18,20.68,22.04,23.30)
lightAnalyzeSimulations(forecast, trials = 100, numRuns = 100)


Light Statistical Analysis (100 runs):
Original forecast total: 338.38

POISSON RESULTS:
  Average total demand: 337.36
  Average absolute error: 53.70

BINOMIAL RESULTS:
  Average total demand: 342.56
  Average absolute error: 47.20

COMPARISON:
  Binomial more accurate: 72/100 times (72.0%)
  Poisson more accurate: 28/100 times (28.0%)
  Winner: Binomial (lower average error)
